# OpenAI Function Calling In LangChain

In [27]:
import os
from dotenv import load_dotenv, find_dotenv

from gigachat import GigaChat
from gigachat.models import Chat, Messages, MessagesRole, chat_completion

from langchain_gigachat.chat_models import GigaChat

_ = load_dotenv(find_dotenv())

api_key  = os.getenv('GIGACHAT_API_KEY')

model = GigaChat(credentials=api_key, verify_ssl_certs=False)

In [2]:
from typing import List
from pydantic import BaseModel, Field

## Синтаксис библиотеки 'Pydantic' (Pydantic Syntax)

Pydantic классы данных это смешение классов из Python вместе с силой валидации из Pydantic

Они предлагают краткий способ определения структур данных, гарантируя при этом соответствие данных заданным типам и ограничениям

В классическом питоне вы можете создать класс, наподобии этого:

<span style="color:grey">Pydantic data classes are a blend of Python's data classes with the validation power of Pydantic.</span>

<span style="color:grey">They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.</span>

<span style="color:grey">In standard python you would create a class like this:</span>

In [3]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [4]:
foo = User(name="Joe",age=32, email="joe@gmail.com")

In [5]:
foo.name

'Joe'

In [6]:
foo = User(name="Joe",age="bar", email="joe@gmail.com")

In [7]:
foo.age

'bar'

In [8]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [9]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [10]:
foo_p.name

'Jane'

<p style=\"background-color:#F5C780; padding:15px\"><b>Note:</b> The next line is expected to fail.</p>

In [11]:
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

ValidationError: 1 validation error for pUser
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='bar', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/int_parsing

In [12]:
class Class(BaseModel):
    students: List[pUser]

In [13]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [14]:
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

## Pydantic to OpenAI function definition


In [15]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get weather for")

In [16]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function

In [17]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

C:\Users\Maksim\AppData\Local\Temp\ipykernel_17176\1313296718.py:1: LangChainDeprecationWarning: The function `_convert_pydantic_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  weather_function = convert_pydantic_to_openai_function(WeatherSearch)


In [18]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [19]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

<p style=\"background-color:#F5C780; padding:15px\"><b>Note:</b> The next cell is expected to generate an error.</p>

In [20]:
convert_pydantic_to_openai_function(WeatherSearch1)

{'name': 'WeatherSearch1',
 'description': '',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [21]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [22]:
convert_pydantic_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [31]:
model.invoke("Какая сегодня погода в аэропорту Санкт-Петербурга", 
                functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': {'airport_code': 'LED'}}, 'functions_state_id': '304a130f-3fc2-4721-aae4-5924a2589199'}, response_metadata={'token_usage': {'prompt_tokens': 91, 'completion_tokens': 29, 'total_tokens': 120}, 'model_name': 'GigaChat:1.0.26.20', 'finish_reason': 'function_call'}, id='run-b1a9c09c-876d-4e74-bafc-35233462a849-0', tool_calls=[{'name': 'WeatherSearch', 'args': {'airport_code': 'LED'}, 'id': '0e26a9a1-c8c3-451a-9d2a-802311c2e46d', 'type': 'tool_call'}])

In [33]:
model_with_function = model.bind(functions=[weather_function])

In [34]:
model_with_function.invoke("Какая сегодня погода в аэропорту Санкт-Петербурга")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': {'airport_code': 'LED'}}, 'functions_state_id': '766442ed-6a8b-4443-921f-21d2615aabc0'}, response_metadata={'token_usage': {'prompt_tokens': 91, 'completion_tokens': 29, 'total_tokens': 120}, 'model_name': 'GigaChat:1.0.26.20', 'finish_reason': 'function_call'}, id='run-349f28cc-c09f-455f-96e9-59bbd99fc7ad-0', tool_calls=[{'name': 'WeatherSearch', 'args': {'airport_code': 'LED'}, 'id': 'd32b76f2-3c1c-4cac-a74a-a3af3c0a3e6c', 'type': 'tool_call'}])

## Forcing it to use a function

We can force the model to use a function

In [35]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"})

In [37]:
model_with_forced_function.invoke("Какая погода в аэропорту СПБ?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': {'airport_code': 'LED'}}, 'functions_state_id': '0d3d2085-22e9-4594-8f59-ba42fd270f5a'}, response_metadata={'token_usage': {'prompt_tokens': 90, 'completion_tokens': 29, 'total_tokens': 119}, 'model_name': 'GigaChat:1.0.26.20', 'finish_reason': 'function_call'}, id='run-fd5d6c55-1e3c-45c5-8d23-8a1a95579a36-0', tool_calls=[{'name': 'WeatherSearch', 'args': {'airport_code': 'LED'}, 'id': 'd9c9b9d2-a234-4a66-9706-469e5d46d1dd', 'type': 'tool_call'}])

In [38]:
model_with_forced_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': {'airport_code': 'MOW'}}, 'functions_state_id': '7aa4062f-eca6-4079-9e5e-41ed61c2ddb1'}, response_metadata={'token_usage': {'prompt_tokens': 82, 'completion_tokens': 30, 'total_tokens': 112}, 'model_name': 'GigaChat:1.0.26.20', 'finish_reason': 'function_call'}, id='run-7b8842ce-7480-4344-baaa-a3c54f0c7f4f-0', tool_calls=[{'name': 'WeatherSearch', 'args': {'airport_code': 'MOW'}, 'id': 'ee3fbfdb-8e59-4fd4-a550-12b94e3f1e3d', 'type': 'tool_call'}])

## Using in a chain

Мы можем использовать эту модель, связанную с функционированием в цепочке, как обычно

<span style="color:grey">We can use this model bound to function in a chain as we normally would</span>

In [39]:
from langchain.prompts import ChatPromptTemplate

In [40]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Вы очень полезный помощник"),
    ("user", "{input}")
])

In [41]:
chain = prompt | model_with_function

In [43]:
chain.invoke({"input": "Какая погода в аэропорту СПБ?"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': {'airport_code': 'LED'}}, 'functions_state_id': '65c16a3a-e761-4911-ab63-b9771104c2ab'}, response_metadata={'token_usage': {'prompt_tokens': 98, 'completion_tokens': 29, 'total_tokens': 127}, 'model_name': 'GigaChat:1.0.26.20', 'finish_reason': 'function_call'}, id='run-3a2b602e-889f-48b2-bb38-beed14bf5345-0', tool_calls=[{'name': 'WeatherSearch', 'args': {'airport_code': 'LED'}, 'id': '09ec71f8-5424-451a-aff9-fe09cdd48410', 'type': 'tool_call'}])

## Using multiple functions

Еще лучше, если мы можем передать набор функций и позволить LLM решить, какую из них использовать, исходя из контекста вопроса.

<span style="color:gray">Even better, we can pass a set of function and let the LLM decide which to use based on the question context.</span>

In [44]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [45]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

In [46]:
model_with_functions = model.bind(functions=functions)

In [47]:
model_with_functions.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'WeatherSearch', 'arguments': {'airport_code': 'SFO'}}, 'functions_state_id': '3a9f3c3f-4c90-4015-8ca4-e5a639f31a12'}, response_metadata={'token_usage': {'prompt_tokens': 166, 'completion_tokens': 30, 'total_tokens': 196}, 'model_name': 'GigaChat:1.0.26.20', 'finish_reason': 'function_call'}, id='run-d9b0a7c5-db8d-4493-b453-884b7ea9d6d6-0', tool_calls=[{'name': 'WeatherSearch', 'args': {'airport_code': 'SFO'}, 'id': 'a6b76f36-b59d-4b67-848d-5401c08b48be', 'type': 'tool_call'}])

In [50]:
model_with_functions.invoke("what are three songs by taylor swift?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'ArtistSearch', 'arguments': {'artist_name': 'Taylor Swift', 'n': 3}}, 'functions_state_id': 'd8b30766-603c-4e40-a1b4-0d85f53ae59f'}, response_metadata={'token_usage': {'prompt_tokens': 168, 'completion_tokens': 37, 'total_tokens': 205}, 'model_name': 'GigaChat:1.0.26.20', 'finish_reason': 'function_call'}, id='run-1b53d4e2-58ea-4b09-ace5-03a7484c1c3c-0', tool_calls=[{'name': 'ArtistSearch', 'args': {'artist_name': 'Taylor Swift', 'n': 3}, 'id': '8db72428-2d07-4883-af2c-998896a43920', 'type': 'tool_call'}])

In [51]:
model_with_functions.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'functions_state_id': 'c4e2463c-c6d2-4102-9fbe-5a6184a80b01'}, response_metadata={'token_usage': {'prompt_tokens': 161, 'completion_tokens': 14, 'total_tokens': 175}, 'model_name': 'GigaChat:1.0.26.20', 'finish_reason': 'stop'}, id='run-367c6c17-c2f0-468f-a2ca-8fc38a01ae76-0')